# Import Necessary Libraries

In [201]:
# import the library we use to open URLs
import urllib.request
import requests

# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup

#import pandas
import pandas as pd
import numpy as np

#import matplotlib and related
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

#import Sci-kit Learn
from sklearn.cluster import KMeans
from collections import Counter

import re

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


#!conda install -c conda-forge folium=0.5.0 --yes 

import folium # map rendering library

#  1. Process Data

## Scrape data from Wikipedia for the Postal Codes

In [49]:
# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")


## Process the data scrapped to create a Pandas Dataframe

In [62]:
toronto_table=soup.find('table', class_='wikitable sortable')

PostalCode=[]
Borough=[]
Neighborhood=[]

for row in toronto_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        PostalCode.append(cells[0].find(text=True))
        Borough.append(cells[1].find(text=True))
        Neighborhood.append(cells[2].find(text=True))

# Create new dataframe and remove special characters on Postal Code 
        
df=pd.DataFrame(PostalCode,columns=['Postal Code'])
df['Postal Code']=df['Postal Code'].map(lambda x: re.sub(r'\W+', '', x))
df['Borough']=Borough
df['Neighborhood']=Neighborhood


# Remove special characters as filter condition was not working to drop rows where Borough was 'Not Assigned'
# Drop rows for Borough is Not Assigned

df['Borough_Filter'] = df['Borough'].map(lambda x: re.sub(r'\W+', '', x))
indexNames =df[df.Borough_Filter == 'Notassigned'].index
df.drop(indexNames , inplace=True)
df.drop(columns=['Borough_Filter'], inplace=True)
df.index = range(len(df))
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


## Shape of the dataframe

In [72]:
df.shape

(103, 3)

# 2. Add Lattitude and Longitude for PostalCode

## Import File with Geo Data as Geocode is not working

In [202]:
url_loc='http://cocl.us/Geospatial_data'
df_loc=pd.read_csv(url_loc)
df_loc

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [228]:
toronto_data = pd.merge(df,
                 df_loc[['Postal Code', 'Latitude', 'Longitude']],
                 on='Postal Code', 
                 how='left')
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


# 3. Explore Neighborhoods in Toronto,CA


In [157]:
CLIENT_ID = '2U31GRV45UN3O0X35QJDCYI3XJGASIIRRPIBKJM3HV4LDW4T' # your Foursquare ID
CLIENT_SECRET = 'A0II252LMARIHXG2MB0XNZNC41W225QI5AKD2GV5HY0SOQQQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2U31GRV45UN3O0X35QJDCYI3XJGASIIRRPIBKJM3HV4LDW4T
CLIENT_SECRET:A0II252LMARIHXG2MB0XNZNC41W225QI5AKD2GV5HY0SOQQQ


In [231]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods
 are 43.7532586, -79.3296565.


## Get  locations around the Neigbourhood

In [209]:
# type your answer here
radius= 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION,  radius, LIMIT)


In [232]:
import requests
results = requests.get(url).json()


In [236]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [237]:
# type your answer here

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )



Parkwoods

Victoria Village

Regent Park / Harbourfront

Lawrence Manor / Lawrence Heights

Queen's Park / Ontario Provincial Government

Islington Avenue

Malvern / Rouge

Don Mills

Parkview Hill / Woodbine Gardens

Garden District, Ryerson

Glencairn

West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale

Rouge Hill / Port Union / Highland Creek

Don Mills

Woodbine Heights

St. James Town

Humewood-Cedarvale

Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood

Guildwood / Morningside / West Hill

The Beaches

Berczy Park

Caledonia-Fairbanks

Woburn

Leaside

Central Bay Street

Christie

Cedarbrae

Hillcrest Village

Bathurst Manor / Wilson Heights / Downsview North

Thorncliffe Park

Richmond / Adelaide / King

Dufferin / Dovercourt Village

Scarborough Village

Fairview / Henry Farm / Oriole

Northwood Park / York University

East Toronto

Harbourfront East / Union Station / Toronto Islands

Little Portugal / Trinity

Kennedy Park / Ionview / E

In [245]:
print(toronto_venues.shape)
toronto_venues.head()

(2129, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [239]:
toronto_venues.groupby(['Neighborhood','Venue']).count()

Neighborhood Latitude  \
Neighborhood                                       Venue                                                          
Agincourt                                                                                                     1   
                                         El Pulgarcito                                                        1   
                                                   Panagio's Breakfast & Lunch                                1   
                                                   Twilight                                                   1   
Alderwood / Long Branch                                                                                       1   
                           Il Paesano Pizzeria & Restaurant                                                   1   
                                                   Pizza Pizza                                                1   
                                                   Rexall                                                     1   
                                                   Sir Adam Beck Rink                                         1   
                                                   Subway                                                     1   
                                                   Tim Hortons                                                1   
                                                   Timothy's Pub                                              1   
                                                   Toronto Gymnastics International                           1   
Bathurst Manor / Wilson Heights / Downsview North                                                             1   
 Bagel Plus                                                                                                   1   
                                                   Best for Bride in Toronto                                  1   
                                                   Dairy Queen                                                1   
                                                   Metro                                                      1   
                                                   Orly Restaurant & Grill                                    1   
                                                   Pet Valu                                                   1   
                                                   Petro-Canada                                               1   
                                                   Pizza Pizza                                                1   
                                                   Popeyes Louisiana Kitchen                                  1   
                                                   RBC Royal Bank                                             1   
                                                   Rogers                                                     1   
                                                   Sheppard Plaza                                             1   
                                                   Sheppard Plaza Variety                                     1   
                                                   Shoppers Drug Mart                                         1   
                                                   Starbucks                                                  1   
                                                   Subway                                                     1   
                                                   TD Canada Trust                                            1   
                                                   Tim Hortons                                                1   
                                                   Wakame Sushi                                               1   
Willowdale                                                       

In [240]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 269 uniques categories.


# 4. Analyze Neighborhood

In [241]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [244]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(10)

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,Alderwood / Long Branch,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,Bathurst Manor / Wilson Heights / Downsview No...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.047619,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,Bedford Park / Lawrence Manor East,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
5,Berczy Park,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.017241,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
6,Birch Cliff / Cliffside West,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
7,Brockton / Parkdale Village / Exhibition Place,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
8,Business reply mail Processing CentrE,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
9,CN Tower / King and Spadina / Railway Lands / ...,0.0,0.0,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


#### Let's print each neighborhood along with the top 5 most common venues

In [247]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt
----
                       venue  freq
0                     Lounge  0.33
1  Latin American Restaurant  0.33
2             Breakfast Spot  0.33
3                Men's Store  0.00
4          Mobile Phone Shop  0.00


----Alderwood / Long Branch
----
            venue  freq
0     Pizza Place  0.25
1        Pharmacy  0.12
2             Pub  0.12
3  Sandwich Place  0.12
4     Coffee Shop  0.12


----Bathurst Manor / Wilson Heights / Downsview North
----
         venue  freq
0  Coffee Shop  0.10
1         Bank  0.10
2     Pharmacy  0.05
3  Gas Station  0.05
4    Pet Store  0.05


----Bayview Village
----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3  Japanese Restaurant  0.25
4          Yoga Studio  0.00


----Bedford Park / Lawrence Manor East
----
                venue  freq
0         Pizza Place  0.08
1         Coffee Shop  0.08
2          Restaurant  0.08
3      Sandwich Place  0.08
4  Italian Restaur

In [248]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [251]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Breakfast Spot,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Drugstore
1,Alderwood / Long Branch,Pizza Place,Gym,Skating Rink,Pharmacy,Pub,Sandwich Place,Coffee Shop,Airport Service,College Gym,Ethiopian Restaurant
2,Bathurst Manor / Wilson Heights / Downsview No...,Bank,Coffee Shop,Pharmacy,Supermarket,Deli / Bodega,Sushi Restaurant,Restaurant,Middle Eastern Restaurant,Convenience Store,Fried Chicken Joint
3,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Women's Store
4,Bedford Park / Lawrence Manor East,Coffee Shop,Pizza Place,Restaurant,Sandwich Place,Italian Restaurant,Sushi Restaurant,Comfort Food Restaurant,Cosmetics Shop,Pharmacy,Café


# 5. Define Clusters
## Find Co-Ordinates for City of Toronto, CA

In [218]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


# Create Map of Toronto

In [71]:
import folium # map rendering library


# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Run k-means to cluster

In [267]:


# set number of clusters

kclusters = 5

toronto_clustering = result.drop(['Neighborhood','Borough','Postal Code'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
toronto_clustering.insert(0, 'Cluster Labels', kmeans.labels_)



In [268]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon,  cluster in zip(toronto_clustering['Latitude'], toronto_clustering['Longitude'], toronto_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters